In [19]:
import ccxt
import pandas as pd
import Metriche_di_performance as mdp 
import time

In [20]:
def mappatura_variazione_acquisti(variazione_perc, A_a, B_a):
    return variazione_perc ** A_a / (B_a + (1 - B_a) * variazione_perc ** A_a)

# Funzione per determinare quanto acquistare
def quanto_acquistare(price, first_buy_price, last_buy_purchase, cash, do_buy, A_a, B_a):
    variazione_primo = abs((price - first_buy_price) / first_buy_price)
    variazione_mappata = mappatura_variazione_acquisti(variazione_primo, A_a, B_a)
    if do_buy:
        return min(last_buy_purchase + variazione_mappata * cash, cash)
    elif last_buy_purchase + variazione_mappata * cash > cash:
        return 0
    else:
        return last_buy_purchase + variazione_mappata * cash

In [21]:
def trading_doppio_TS_6(dati, initial_cash, initial_bitcoin, commission_buy_pct, commission_sell_pct, min_buy, min_sell, first_buy_pct, do_buy, min_earn_pct, buy_only_under_last_price, A_a, B_a):
    """
    Simula una strategia semplice di acquisti e vendite a mercato.

    Args:
        dati (pd.DataFrame): DataFrame con almeno due colonne: 'Timestamp' e 'Price'.
        initial_cash (float): Liquidità iniziale (in USDT).
        initial_bitcoin (float): Quantità iniziale di bitcoin.
        commission_buy_pct (float): Percentuale di commissione sull'acquisto (esempio: 0.001 per 0.1%).
        commission_sell_pct (float): Percentuale di commissione sulla vendita (esempio: 0.001 per 0.1%).
        min_buy (float): Quantità minima in USDT per effettuare un acquisto.
        min_sell (float): Quantità minima in USDT per effettuare una vendita.
        first_buy_pct (float): Percentuale della liquidità da usare per il primo acquisto di ogni ciclo.
        do_buy (bool): Se True, usa tutta la liquidità quando è richiesta più liquidità di quella posseduta, altrimenti non acquista niente.
        min_earn_pct (float): Percentuale di guadagno minimo che si desidera tra vendita e prezzo medio pesato degli acquisti.
        A_a (float): Parametro per la funzione di mappatura della variazione.
        B_a (float): Parametro per la funzione di mappatura della variazione.

    Returns:
        pd.DataFrame: DataFrame delle transazioni giornaliere con dettagli.
    """

    # Variabili di supporto
    cash = initial_cash
    bitcoin = initial_bitcoin
    prezzo_medio_pesato_acquisti = None
    # last_buy_purchase = 0  # Liquidità spesa per l'ultimo acquisto
    # total_purchase = 0
    # first_buy_price = None
    last_action = None  # Ultima azione effettiva: "Acquisto" o "Vendita"

    transactions = []  # Lista per registrare le transazioni giornaliere

    for i, row in dati.iterrows():
        price = row['Price']
        timestamp = row['Timestamp']

        # Variabili per liquidità spesa o guadagnata e bitcoin comprati o venduti
        liquidità_spesa_o_guadagnata = 0
        bitcoin_comprati_o_venduti = 0

        # Azione del giorno, "-" se nessuna azione
        action = "-"

        if last_action is None or last_action == "Vendita":
            # Primo acquisto del ciclo
            liquidità_da_usare = cash * first_buy_pct
            if liquidità_da_usare >= min_buy:
                bitcoin_bought = liquidità_da_usare / price * (1 - commission_buy_pct)
                bitcoin += bitcoin_bought
                liquidità_spesa_o_guadagnata = liquidità_da_usare
                cash -= liquidità_da_usare
                prezzo_medio_pesato_acquisti = price
                last_buy_purchase = liquidità_da_usare
                total_purchase = liquidità_da_usare
                first_buy_price = price
                last_buy_price = price
                last_action = "Acquisto"
                action = "Primo acquisto"

        elif last_action == "Acquisto":
            # Calcola la soglia per un nuovo acquisto
            soglia = prezzo_medio_pesato_acquisti / ((1 - commission_buy_pct) * (1 - min_earn_pct))

            if price <= soglia:
                # Effettua un ulteriore acquisto
                liquidità_da_usare = quanto_acquistare(price, first_buy_price, last_buy_purchase, cash, do_buy, A_a, B_a)
                if liquidità_da_usare >= min_buy and (not buy_only_under_last_price or buy_only_under_last_price and price < last_buy_price):
                    bitcoin_bought = liquidità_da_usare / price * (1 - commission_buy_pct)
                    bitcoin += bitcoin_bought
                    liquidità_spesa_o_guadagnata = liquidità_da_usare
                    total_purchase += liquidità_da_usare
                    cash -= liquidità_da_usare
                    prezzo_medio_pesato_acquisti = ((prezzo_medio_pesato_acquisti * (total_purchase - liquidità_da_usare)) +
                                                    (price * liquidità_da_usare)) / total_purchase
                    last_buy_purchase = liquidità_da_usare
                    last_buy_price = price
                    action = "Acquisto"

            else:
                # Vendi tutti i bitcoin posseduti
                liquidità_spesa_o_guadagnata = bitcoin * price * (1 - commission_sell_pct)
                if liquidità_spesa_o_guadagnata >= min_sell:
                    bitcoin_comprati_o_venduti = bitcoin
                    cash += liquidità_spesa_o_guadagnata
                    bitcoin = 0
                    last_action = "Vendita"
                    action = "Vendita"

                    # Resetta le variabili dopo la vendita
                    # prezzo_medio_pesato_acquisti = None
                    # last_buy_purchase = 0
                    # first_buy_price = None
                    # total_purchase = 0
                    # last_buy_price = None

        # Calcola il valore del portafoglio alla fine della giornata
        portfolio_value = cash + bitcoin * price

        # Registra la riga giornaliera
        transactions.append({
            "timestamp": timestamp, "prezzo": price, "azione": action, "ultima_azione": last_action or "-",
            "prezzo_medio_pesato_acquisti": prezzo_medio_pesato_acquisti, "liquidità_spesa_o_guadagnata": liquidità_spesa_o_guadagnata,
            "bitcoin_comprati_o_venduti": bitcoin_comprati_o_venduti, "cash": cash, "bitcoin": bitcoin,
            "valore_totale_portafoglio": portfolio_value
        })

    # Crea un DataFrame dalle transazioni
    transactions_df = pd.DataFrame(transactions)
    return transactions_df

In [31]:
# Parametri configurabili
L = 30  # Numero di giorni
TIMEFRAME = '6h'  # Timeframe ('1m', '5m', '1h', '1d', etc.)
MARKET = 'bybit'
initial_cash = 100
initial_bitcoin = 0
commission_buy_pct = 0.001 # 0.1% commissione acquisto
commission_sell_pct = 0.001 # 0.1% commissione vendita
min_buy = 5 # Prezzo minimo di acquisto
min_sell = 5 # Prezzo minimo di vendita
first_buy_pct = 0.1
do_buy = True # Sarebbe il compra_pure, che se è True e l'acquisto è superiore alla liquidità disponibile, usa tutta la liquidità, altrimenti non fa niente
min_earn_pct = 0.02
buy_only_under_last_price = False # Se True, compra solo se il prezzo è inferiore al prezzo dell'ultimo acquisto, altrimenti può comprare comunque
A_a = 1
B_a = 1

# Inizializza l'exchange
exchange = ccxt.__getattribute__(MARKET)()

def fetch_crypto_data(exchange, symbol, timeframe, since):
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since)
        df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
        return df[['Timestamp', 'Close']]
    except Exception as e:
        print(f"Errore nel fetching dei dati per {symbol}: {e}")
        return None

In [32]:
# Calcola il timestamp di inizio per L giorni fa
since = int((time.time() - L * 24 * 60 * 60) * 1000)

# Ottieni i mercati disponibili sull'exchange specificato
markets = exchange.load_markets()
symbols = [symbol for symbol in markets if symbol.endswith('/USDT')]  # Filtra per coppie USDT

rendimenti = []
metriche_per_crypto = {}

for symbol in symbols:
    print(f"Scaricando i dati per {symbol}...")
    df = fetch_crypto_data(exchange, symbol, TIMEFRAME, since)
    if df is None or df.empty:
        continue

    # Aggiungi una colonna 'Price' per compatibilità con la tua strategia
    df.rename(columns={'Close': 'Price'}, inplace=True)

    # Applica la tua strategia per ottenere le transazioni giornaliere
    df_transazioni = trading_doppio_TS_6(df, initial_cash, initial_bitcoin, commission_buy_pct, commission_sell_pct, min_buy, min_sell, first_buy_pct, do_buy, min_earn_pct, buy_only_under_last_price, A_a, B_a)  # Sostituisci con la tua funzione

    indice_ultima_vendita = df_transazioni[df_transazioni['azione'] == 'Vendita']['timestamp'].idxmax() if "Vendita" in df_transazioni['azione'].unique() else None
    if indice_ultima_vendita is not None:
        df_transazioni_tagliato_ad_ultima_vendita = df_transazioni[:indice_ultima_vendita+1]
    else:
        df_transazioni_tagliato_ad_ultima_vendita = df_transazioni.copy()

    # Calcola il rendimento del portafoglio
    valore_totale_finale = df_transazioni_tagliato_ad_ultima_vendita['valore_totale_portafoglio'].iloc[-1]
    rendimento = valore_totale_finale / (initial_cash + initial_bitcoin * df['Price'].iloc[0])
    rendimenti.append(rendimento)

    # Calcola metriche usando la funzione "calcola_tutte_le_metriche" di mdp
    metriche = mdp.calcola_principali_metriche(
        prices=df_transazioni_tagliato_ad_ultima_vendita['prezzo'],
        bitcoin_posseduti=df_transazioni_tagliato_ad_ultima_vendita['bitcoin'],
        portfolio_values=df_transazioni_tagliato_ad_ultima_vendita['valore_totale_portafoglio']
    )
    metriche_per_crypto[symbol] = metriche

# Calcola la media dei rendimenti
if rendimenti:
    media_rendimento = sum(rendimenti) / len(rendimenti)
    print(f"\nMedia dei fattori di rendimento su tutte le crypto: {media_rendimento:.2f}")
else:
    print("\nNessun rendimento calcolato.")

# Trova la crypto con valore massimo e minimo per ogni metrica
if metriche_per_crypto:
    df_metriche = pd.DataFrame(metriche_per_crypto).T
    for metrica in df_metriche.columns:
        crypto_max = df_metriche[metrica].idxmax()
        crypto_min = df_metriche[metrica].idxmin()
        print(f"\nPer la metrica '{metrica}':")
        print(f"  Crypto con valore massimo: {crypto_max} ({df_metriche[metrica].max():.2f})")
        print(f"  Crypto con valore minimo: {crypto_min} ({df_metriche[metrica].min():.2f})")

Scaricando i dati per BTC/USDT...
Scaricando i dati per ETH/USDT...
Scaricando i dati per XRP/USDT...
Scaricando i dati per EOS/USDT...
Scaricando i dati per DOT/USDT...
Scaricando i dati per XLM/USDT...
Scaricando i dati per LTC/USDT...
Scaricando i dati per DOGE/USDT...
Scaricando i dati per CHZ/USDT...
Scaricando i dati per AXS/USDT...
Scaricando i dati per MANA/USDT...
Scaricando i dati per DYDX/USDT...
Scaricando i dati per MKR/USDT...
Scaricando i dati per COMP/USDT...
Scaricando i dati per AAVE/USDT...
Scaricando i dati per YFI/USDT...
Scaricando i dati per LINK/USDT...
Scaricando i dati per SUSHI/USDT...
Scaricando i dati per UNI/USDT...
Scaricando i dati per KSM/USDT...
Scaricando i dati per ICP/USDT...
Scaricando i dati per ADA/USDT...
Scaricando i dati per ETC/USDT...
Scaricando i dati per XTZ/USDT...
Scaricando i dati per BCH/USDT...
Scaricando i dati per QNT/USDT...
Scaricando i dati per USDC/USDT...


c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Scaricando i dati per GRT/USDT...
Scaricando i dati per SOL/USDT...
Scaricando i dati per FIL/USDT...
Scaricando i dati per OMG/USDT...
Scaricando i dati per BAT/USDT...
Scaricando i dati per ZRX/USDT...
Scaricando i dati per CRV/USDT...
Scaricando i dati per AGLD/USDT...
Scaricando i dati per ANKR/USDT...
Scaricando i dati per PERP/USDT...
Scaricando i dati per WAVES/USDT...
Scaricando i dati per LUNC/USDT...
Scaricando i dati per SPELL/USDT...
Scaricando i dati per SHIB/USDT...
Scaricando i dati per FTM/USDT...
Scaricando i dati per ATOM/USDT...
Scaricando i dati per ALGO/USDT...
Scaricando i dati per ENJ/USDT...
Scaricando i dati per CBX/USDT...
Scaricando i dati per SAND/USDT...
Scaricando i dati per AVAX/USDT...
Scaricando i dati per WOO/USDT...
Scaricando i dati per FTT/USDT...
Scaricando i dati per GODS/USDT...
Scaricando i dati per IMX/USDT...
Scaricando i dati per ENS/USDT...
Scaricando i dati per CAKE/USDT...
Scaricando i dati per STETH/USDT...
Scaricando i dati per GALFT/USD

c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Scaricando i dati per ACA/USDT...
Scaricando i dati per MV/USDT...
Scaricando i dati per MIX/USDT...
Scaricando i dati per RSS3/USDT...
Scaricando i dati per TAP/USDT...
Scaricando i dati per ERTHA/USDT...
Scaricando i dati per GMX/USDT...
Scaricando i dati per T/USDT...
Scaricando i dati per ACH/USDT...
Scaricando i dati per JST/USDT...
Scaricando i dati per SUN/USDT...
Scaricando i dati per BTT/USDT...
Scaricando i dati per TRX/USDT...
Scaricando i dati per NFT/USDT...
Scaricando i dati per POKT/USDT...
Scaricando i dati per SCRT/USDT...
Scaricando i dati per PSTAKE/USDT...
Scaricando i dati per SON/USDT...
Scaricando i dati per HERO/USDT...
Scaricando i dati per DOME/USDT...
Scaricando i dati per USTC/USDT...
Scaricando i dati per BNB/USDT...
Scaricando i dati per NEAR/USDT...
Scaricando i dati per PAXG/USDT...


...

c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Scaricando i dati per DEEP/USDT...
Scaricando i dati per PUFFER/USDT...
Scaricando i dati per DBR/USDT...
Scaricando i dati per PUFF/USDT...
Scaricando i dati per SCR/USDT...
Scaricando i dati per X/USDT...
Scaricando i dati per COOK/USDT...
Scaricando i dati per GRASS/USDT...
Scaricando i dati per KRO/USDT...
Scaricando i dati per SMILE/USDT...
Scaricando i dati per KAIA/USDT...
Scaricando i dati per SWELL/USDT...
Scaricando i dati per BBQ/USDT...
Scaricando i dati per NS/USDT...
Scaricando i dati per GOAT/USDT...
Scaricando i dati per CMETH/USDT...
Scaricando i dati per MORPHO/USDT...
Scaricando i dati per NEIROCTO/USDT...
Scaricando i dati per STOP/USDT...
Scaricando i dati per BAN/USDT...


c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Scaricando i dati per OL/USDT...
Scaricando i dati per VIRTUAL/USDT...
Scaricando i dati per MAJOR/USDT...
Scaricando i dati per MEMEFI/USDT...
Scaricando i dati per PNUT/USDT...
Scaricando i dati per SPX/USDT...
Scaricando i dati per THRUST/USDT...
Scaricando i dati per ZRC/USDT...
Scaricando i dati per LUCE/USDT...
Scaricando i dati per CHILLGUY/USDT...
Scaricando i dati per SUPRA/USDT...
Scaricando i dati per PAAL/USDT...
Scaricando i dati per HPOS10I/USDT...

Media dei fattori di rendimento su tutte le crypto: 1.42

Per la metrica 'Total Return':
  Crypto con valore massimo: SCRT/USDT (138.46)
  Crypto con valore minimo: ETH3S/USDT (-75.93)

Per la metrica 'Max Drawdown':
  Crypto con valore massimo: ETH3S/USDT (76.07)
  Crypto con valore minimo: LIS/USDT (0.00)

Per la metrica 'Volatility':
  Crypto con valore massimo: STOP/USDT (3.38)
  Crypto con valore minimo: LIS/USDT (0.00)

Per la metrica 'Sharpe Ratio':
  Crypto con valore massimo: LIS/USDT (inf)
  Crypto con valore minimo:

c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\alessandro.imbriaco\Desktop\myenv\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
